<a href="https://colab.research.google.com/github/PLSTELLMEIAMNOTUNDONE/ScheduleProblem/blob/master/schedule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install ortools



In [508]:
from ortools.sat.python import cp_model

In [911]:
lecture_rooms = 0
casual_rooms = 5
teachers = 15
subjects = 6
lessons = 30
casual_groups = 9
lecture_groups = 0
subjectGroup = [
                [2, 2, 2, 2, 2, 2, 0, 0, 5],
                [0, 0, 0, 0, 1, 1, 4, 1, 1],
                [1, 2, 1, 1, 1, 2, 0, 0, 0],
                [0, 0, 0, 4, 0, 1, 1, 1, 1],
                [0, 2, 1, 1, 0, 0, 2, 1, 1],
                [0, 0, 2, 1, 1, 1, 1, 2, 1]
                ]
subGroup = {}

unity = {}
for g in range(casual_groups + lecture_groups):
    unity[g] = [g]
    for lg, v in subGroup.items():
        if g in v:
            unity[g].append(lg)
print(unity)
lecture_rooms_range = range(casual_rooms, lecture_rooms + casual_rooms)
casual_rooms_range = range(casual_rooms)
all_subjects = range(subjects)
all_lessons = range(lessons)
lecture_groups_range = range(casual_groups, lecture_groups + casual_groups)
casual_groups_range = range(casual_groups)
all_groups = range(lecture_groups + casual_groups)
all_rooms = range(lecture_rooms + casual_rooms)
all_teachers = range(teachers)
class_sum = 0
for s in all_subjects:
    for g in all_groups:
        class_sum += subjectGroup[s][g]
print(class_sum)

roomsNames = {}
subjectsNames = {}
lessonsNames = {}
groupsNames = {}
teachersNames = {}
for r in casual_rooms_range:
    roomsNames[r] = "cr" + str(r + 1)
for r in lecture_rooms_range:
    roomsNames[r] = "lr" + str(r + 1)
for s in all_subjects:
    subjectsNames[s] = "s" + str(s + 1)
for l in all_lessons:
    lessonsNames[l] = "l" + str(l + 1)
for g in casual_groups_range:
    groupsNames[g] = "g" + str(g + 1)
for g in lecture_groups_range:
    groupsNames[g] = str([groupsNames[s] for s in subGroup[g]])
for t in all_teachers:
    teachersNames[t] = "t" + str(t + 1)
subjectGroupMap = {}

for s in all_subjects:
    for g in all_groups:
        if subjectGroup[s][g] != 0:
            subjectGroupMap[(s, g)] = subjectGroup[s][g]



{0: [0], 1: [1], 2: [2], 3: [3], 4: [4], 5: [5], 6: [6], 7: [7], 8: [8]}
58


In [927]:
def sch():

    schedule = {}

    model = cp_model.CpModel()
    for r in casual_rooms_range:
        for s in all_subjects:
            for l in all_lessons:
                for g in casual_groups_range:
                    schedule[(r, s, l, g)] = model.NewBoolVar(
                        f"sch_{roomsNames[r]}_{subjectsNames[s]}_{lessonsNames[l]}_{groupsNames[g]}")
    for r in lecture_rooms_range:
        for s in all_subjects:
            for l in all_lessons:
                for g in lecture_groups_range:
                    schedule[(r, s, l, g)] = model.NewBoolVar(
                        f"sch_{roomsNames[r]}_{subjectsNames[s]}_{lessonsNames[l]}_{groupsNames[g]}")

    for r in lecture_rooms_range:
        for l in all_lessons:
            model.AddAtMostOne(schedule[(r, s, l, g)] for s in all_subjects for g in lecture_groups_range)
    for r in casual_rooms_range:
        for l in all_lessons:
            model.AddAtMostOne(schedule[(r, s, l, g)] for s in all_subjects for g in casual_groups_range)
    # for s in all_subjects:
    #     for l in all_lessons:
    #         model.AddAtMostOne((schedule[(r, s, l, g)] for g in all_groups for r in
    #                             (casual_rooms_range if g in casual_groups_range else lecture_rooms_range)))
    for g in all_groups:
        for l in all_lessons:
            model.AddAtMostOne((schedule[(r, s, l, ug)] for ug in unity[g]
                                for s in all_subjects
                                for r in
                                (casual_rooms_range if ug in casual_groups_range else lecture_rooms_range)))

    for s in all_subjects:
        for g in all_groups:
            model.Add(sum(schedule[(r, s, l, g)]
                          for r in
                          (casual_rooms_range if g in casual_groups_range else lecture_rooms_range)
                          for l in all_lessons) == subjectGroup[s][g])
    # for k, v in subjectGroupMap.items():
    #     s, g = k
    #     model.Add(sum(
    #         schedule[(r, s, l, g)] for r in (casual_rooms_range if g in casual_groups_range else lecture_rooms_range)
    #         for l in all_lessons) == v)

    lessonToOptimize = []
    for l in all_lessons:
        if (l) % 5 > 0 and l != 0:
            lessonToOptimize.append(l)
    indix = {}
    for l in all_lessons:
        for g in casual_groups_range:
            if l % 5 <= 1:
                continue
            d = l // 5
            indix[(l, g)] = model.NewBoolVar(f"ind_{[l]}_{groupsNames[g]}")
            model.AddMaxEquality(indix[(l, g)], [schedule[(r, s, ll, ug)]
                                                 for ll in range(d * 5, l - 1)
                                                 for ug in unity[g]
                                                 for r in (
                                                     casual_rooms_range if ug in casual_groups_range else lecture_rooms_range)
                                                 for s in all_subjects])
            model.Add(indix[(l, g)] - sum(schedule[(r, s, l - 1, ug)]
                                          for ug in unity[g]
                                          for r in (
                                              casual_rooms_range if ug in casual_groups_range else lecture_rooms_range)
                                          for s in all_subjects)
                      + sum(schedule[(r, s, l, ug)]
                            for ug in unity[g]
                            for r in (casual_rooms_range if ug in casual_groups_range else lecture_rooms_range)
                            for s in all_subjects) <= 1)
    # z = sum(sum(schedule[(r, s, l, g)]
    #             for r in (casual_rooms_range if g in casual_groups_range else lecture_rooms_range)
    #             for s in all_subjects) * ((l + 0) % 5)
    #         for g in all_groups
    #         for l in lessonToOptimize)
    # model.Minimize(z)
    # model.Add(z <= 10)

    solver = cp_model.CpSolver()

    solver.parameters.enumerate_all_solutions = False
    status = solver.Solve(model)

    # print(solver.SolutionInfo())
    # print(solver.ResponseStats())

    if status == cp_model.FEASIBLE:
        print("ok")
    if status == cp_model.INFEASIBLE:
        print("not ok")

    print(status)
    result = {}
    sss = 0
    for k, v in schedule.items():
        r, s, l, g = k

        if solver.Value(v) and (s, g) in subjectGroupMap:
            result[k] = 1
            sss += 1



    print(sss)
    # fix!
    def errors(schedule_inst):
        e = 0
        for g in casual_groups_range:
            for d in range(0, 6):
                last = -1
                start = False
                for l in range(0, 5):
                    exist = (any((r, s, (l + d * 5), g) in schedule_inst
                                 for r in casual_rooms_range
                                 for s in all_subjects)
                             or any((r, s, (l + d * 5), lg) in schedule_inst and g in subGroup[lg]
                                    for r in lecture_rooms_range
                                    for s in all_subjects
                                    for lg in lecture_groups_range))

                    if exist and start:
                        if last != l - 1:
                            for i in range(last + 1, l):
                                print("день " + str(d + 1) + " пара " + str(i + 1) + " грyппа " + groupsNames[g])
                                e += 1
                        last = l
                        continue
                    if exist:
                        start = True
                        last = l

        print("errors = " + str(e))


    errors(result)
    return result



In [928]:
def print_sch(result):
  ans = {}
  for l in all_lessons:
      ans[l] = "расписания на день " + str((l // 5) + 1) + " пара № " + str((l % 5) + 1) + "\n"
  for k, v in result.items():
      r, s, l, g = k
      ans[l] += subjectsNames[s] + " проходит y грyппы " + groupsNames[g] + " в kабинете " + roomsNames[r] + "\n"
  for l in all_lessons:
      print(ans[l])

In [929]:
def sch_simple_v_teachers_and_pairs():


    unity = {}
    for g in range(casual_groups + lecture_groups):
        unity[g] = [g]
        for lg, v in subGroup.items():
            if g in v:
                unity[g].append(lg)
    print(unity)

    all_teachers = range(teachers)
    lecture_rooms_range = range(casual_rooms, lecture_rooms + casual_rooms)
    casual_rooms_range = range(casual_rooms)
    all_subjects = range(subjects)
    all_lessons = range(lessons)
    lecture_groups_range = range(casual_groups, lecture_groups + casual_groups)
    casual_groups_range = range(casual_groups)
    all_groups = range(lecture_groups + casual_groups)
    all_rooms = range(lecture_rooms + casual_rooms)

    class_sum = 0
    for s in all_subjects:
        for g in all_groups:
            class_sum += subjectGroup[s][g]
    # print(class_sum)
    subjectGroupMap = {}
    teacherSubjects = {}
    subjectTeachers = {}
    for t in all_teachers:
      teacherSubjects[t] = [s for s in all_subjects]
    for s in all_subjects:

        for g in all_groups:

            if subjectGroup[s][g] != 0:
                subjectGroupMap[(s, g)] = subjectGroup[s][g]
        subjectTeachers[s] = [t for t in all_teachers]

    print(subjectTeachers)
    print(teacherSubjects)



    roomsNames = {}
    subjectsNames = {}
    lessonsNames = {}
    groupsNames = {}
    teachersNames = {}
    for r in casual_rooms_range:
        roomsNames[r] = "cr" + str(r + 1)
    for r in lecture_rooms_range:
        roomsNames[r] = "lr" + str(r + 1)
    for s in all_subjects:
        subjectsNames[s] = "s" + str(s + 1)
    for l in all_lessons:
        lessonsNames[l] = "l" + str(l + 1)
    for g in casual_groups_range:
        groupsNames[g] = "g" + str(g + 1)
    for g in lecture_groups_range:
        groupsNames[g] = str([groupsNames[s] for s in subGroup[g]])
    for t in all_teachers:
        teachersNames[t] = "t" + str(t + 1)

    schedule = {}
    teachers_realm = {}
    model = cp_model.CpModel()
    for r in casual_rooms_range:
        for s in all_subjects:
            for l in all_lessons:
                for g in casual_groups_range:
                  schedule[(r, s, l, g)] = model.NewBoolVar(
                          f"sch_{roomsNames[r]}_{subjectsNames[s]}_{lessonsNames[l]}_{groupsNames[g]}")

    for r in lecture_rooms_range:
        for s in all_subjects:
            for l in all_lessons:
                for g in lecture_groups_range:
                    schedule[(r, s, l, g)] = model.NewBoolVar(
                          f"sch_{roomsNames[r]}_{subjectsNames[s]}_{lessonsNames[l]}_{groupsNames[g]}")
    for g in all_groups:
        for s in all_subjects:
          if subjectGroup[s][g] != 0:
              for t in subjectTeachers[s]:
                  teachers_realm[(g, s, t)] = model.NewBoolVar(f"teach_{roomsNames[r]}_{subjectsNames[s]}_{teachersNames[t]}")

    for r in lecture_rooms_range:
        for l in all_lessons:
            model.AddAtMostOne(schedule[(r, s, l, g)] for s in all_subjects for g in lecture_groups_range)
    for r in casual_rooms_range:
        for l in all_lessons:
            model.AddAtMostOne(schedule[(r, s, l, g)] for s in all_subjects for g in casual_groups_range)
    for g in all_groups:
        for l in all_lessons:
            model.AddAtMostOne((schedule[(r, s, l, ug)] for ug in unity[g]
                                for s in all_subjects
                                for r in
                                (casual_rooms_range if ug in casual_groups_range else lecture_rooms_range))
                               )

    for s in all_subjects:
        for g in all_groups:
            model.Add(sum(schedule[(r, s, l, g)]
                          for r in
                          (casual_rooms_range if g in casual_groups_range else lecture_rooms_range)
                          for l in all_lessons) == subjectGroup[s][g]

                      )


    # teachers constraints
    for s in all_subjects:
        for g in all_groups:
           if subjectGroup[s][g] != 0:
              model.Add(sum(teachers_realm[(g, s, t)] for t in subjectTeachers[s]) == 1)
    # for t in all_teachers:
    #     for l in all_lessons:
    #       relativeS = []
    #       relativeG = []
    #       for s in teacherSubjects[t]:
    #         for g in all_groups:
    #           if subjectGroup[s][g] != 0:
    #             relativeS.append(s)
    #             relativeG.append(g)
    #       b2 = sum(schedule[(r,s,l,g)] for s in relativeS for g in relativeG for r in
    #                   (casual_rooms_range if g in casual_groups_range else lecture_rooms_range))
    #       # print(b2)
    #       model.Add(b2 <= 1)



    # for k, v in subjectGroupMap.items():
    #     s, g = k
    #     model.Add(sum(
    #         schedule[(r, s, l, g)] for r in (casual_rooms_range if g in casual_groups_range else lecture_rooms_range)
    #         for l in all_lessons) == v)

    lessonToOptimize = []
    for l in all_lessons:
        if (l) % 5 > 0 and l != 0:
            lessonToOptimize.append(l)
    indix = {}
    for l in all_lessons:
        for g in casual_groups_range:
            if l % 5 <= 1:
                continue
            d = l // 5
            indix[(l, g)] = model.NewBoolVar(f"ind_{[l]}_{groupsNames[g]}")
            model.AddMaxEquality(indix[(l, g)], [schedule[(r, s, ll, ug)]
                                                 for ll in range(d * 5, l - 1)
                                                 for ug in unity[g]
                                                 for r in (
                                                     casual_rooms_range if ug in casual_groups_range else lecture_rooms_range)
                                                 for s in all_subjects]
                                 )
            model.Add(indix[(l, g)] - sum(schedule[(r, s, l - 1, ug)]
                                          for ug in unity[g]
                                          for r in (
                                              casual_rooms_range if ug in casual_groups_range else lecture_rooms_range)
                                          for s in all_subjects
                                          )
                      + sum(schedule[(r, s, l, ug)]
                            for ug in unity[g]
                            for r in (casual_rooms_range if ug in casual_groups_range else lecture_rooms_range)
                            for s in all_subjects) <= 1)
    # z = sum(sum(schedule[(r, s, l, g)]
    #             for r in (casual_rooms_range if g in casual_groups_range else lecture_rooms_range)
    #             for s in all_subjects) * ((l + 0) % 5)
    #         for g in all_groups
    #         for l in lessonToOptimize)
    # model.Minimize(z)
    # model.Add(z <= 70)

    solver = cp_model.CpSolver()

    solver.parameters.enumerate_all_solutions = False
    status = solver.Solve(model)

    # print(solver.SolutionInfo())
    # print(solver.ResponseStats())

    if status == cp_model.FEASIBLE:
        print("ok")
    if status == cp_model.INFEASIBLE:
        print("not ok")

    print(status)
    result_sch = {}
    sss = 0
    for k, v in schedule.items():
        r, s, l, g = k

        if solver.Value(v) and (s, g) in subjectGroupMap:
            result_sch[k] = 1
            sss += 1
    result_teachers = {}
    sss = 0
    for k, v in teachers_realm.items():
        g, s, t = k

        if solver.Value(v) and (s, g) in subjectGroupMap:
          result_teachers[(g, s)] = t

    ans = {}

    print(sss)
    # fix!
    def errors(schedule_inst):
        e = 0
        for g in casual_groups_range:
            for d in range(0, 6):
                last = -1
                start = False
                for l in range(0, 5):
                    exist = (any((r, s, (l + d * 5), g) in schedule_inst
                                 for r in casual_rooms_range
                                 for s in all_subjects
                                 )
                             or any((r, s, (l + d * 5), lg) in schedule_inst and g in subGroup[lg]
                                    for r in lecture_rooms_range
                                    for s in all_subjects
                                    for lg in lecture_groups_range
                                   )
                    )

                    if exist and start:
                        if last != l - 1:
                            for i in range(last + 1, l):
                                print("день " + str(d + 1) + " пара " + str(i + 1) + " грyппа " + groupsNames[g])
                                e += 1
                        last = l
                        continue
                    if exist:
                        start = True
                        last = l

        print("errors = " + str(e))

    for l in all_lessons:
        ans[l] = "расписания на день " + str((l // 5) + 1) + " пара № " + str((l % 5) + 1) + "\n"
    for k, v in result_sch.items():
        r, s, l, g = k

        ans[l] += subjectsNames[s] + " проходит y грyппы " + groupsNames[g] + " в kабинете " + roomsNames[r] +  " ведет " + teachersNames[result_teachers[(g, s)]] +"\n"
    for l in all_lessons:
        print(ans[l])
    errors(result_sch)




In [930]:
from random import randrange
from random import random
from random import choice
import math
cons = {}
unused_teachers = [t for t in range(teachers)]
teachers_count = [0 for t in range(teachers)]

In [931]:

def temperature(t, i):
  return t / (0.01 * i)
def energy(state, result_sch):
  cons = {}
  bad_coef_window = 1
  bad_coef_same_day = 10
  ans = 0
  last_in_day = {}
  for d in range(6):
    last_in_day[d] = [-1 for t in all_teachers]
  for l in range(lessons):
    for t in range(teachers):
      cons[(t, l)] = 0
  for k, v in result_sch.items():
    r, s, l, g = k
    t = state[(s, g)]
    d = l // 5
    p = l % 5
    cons[(t, l)] += 1
    if cons[(t, l)] > 1:
      ans += bad_coef_same_day * cons[(t, l)]
    if last_in_day[d][t] != -1 and last_in_day[d][t] != p - 1:
      ans += bad_coef_window
    last_in_day[d][t] = p

  return ans

In [932]:
def rand_relatibe_pair():
  return choice(list(subjectGroupMap.keys()))
def puff():
  return
def change_state(state):
  n = randrange(5) + 1
  fix = puff
  while(n > 0):
    n -= 1
    if len(unused_teachers) == 0:
      us, ug = rand_relatibe_pair()
      vs, vg = rand_relatibe_pair()
      state[(us, ug)], state[(vs, vg)] = state[(vs, vg)], state[(us, ug)]
      fix = get_fix(ug, vg, us, vs, state, fix)
    else:
      us, ug = rand_relatibe_pair()
      old_t = state[(us, ug)]
      new_t = unused_teachers.pop()
      state[(us, ug)] = new_t
      teachers_count[new_t] += 1
      teachers_count[old_t] -= 1
      if teachers_count[old_t] == 0:
        unused_teachers.append(old_t)
      fix = get_fix(ug, ug, us, us, state, fix)
  return state, fix
def get_fix(ug, vg, us, vs, state, pref_fix):
  def fix():
    state[(us, ug)], state[(vs, vg)] = state[(vs, vg)], state[(us, ug)]
    pref_fix()

  return fix

In [933]:
def transition(probability):
  bound = random()
  return probability >= bound

In [934]:
def init_state(result_sch, subjectTeachers):
  state = {}
  for g in range(casual_groups):
    for s in range(subjects):
      if subjectGroup[s][g] != 0:
        new_t = randrange(teachers)
        state[(s, g)] = new_t
        if teachers_count[new_t] == 0:
          unused_teachers.remove(new_t)
        teachers_count[new_t] += 1
  return state

In [935]:
def SA_for_teachers(result_sch, subjectTeachers):
  all_teachers = range(teachers)
  temp = 1000000
  state = init_state(result_sch, subjectTeachers)
  E = energy(state, result_sch)
  i = 1
  best_state = state.copy()
  best_E = E
  while temp > 1e-10:
    new_state, fix = change_state(state)

    new_E = energy(new_state, result_sch)
    delt_E = new_E - E
    if delt_E >= 0 and not transition(math.exp(-delt_E / temp)):
      print("fixed")
      fix()
    else:
      print("changed")
      E = new_E
      state = new_state
    if E == 0:
       print("break")
       break
    if E < best_E:
      best_E = E
      best_state = state.copy()
    temp = temperature(temp, i)
    i += 1

    print(f'step: {i}, enegry : {E}, temp : {temp}')
  return best_state


In [936]:

def print_sch_w_teachers(result, state):
    ans = {}
    for l in all_lessons:
        ans[l] = "расписания на день " + str((l // 5) + 1) + " пара № " + str((l % 5) + 1) + "\n"
    for k, v in result.items():
        r, s, l, g = k

        ans[l] += subjectsNames[s] + " проходит y грyппы " + groupsNames[g] + " в kабинете " + roomsNames[r] +  " ведет " + teachersNames[state[(s, g)]] +"\n"
    for l in all_lessons:
        print(ans[l])


In [937]:
result = sch()


4
58
errors = 0


In [938]:
result = dict(sorted(result.items(), key=lambda item: item[0][2]))
print_sch(result)

расписания на день 1 пара № 1

расписания на день 1 пара № 2
s2 проходит y грyппы g5 в kабинете cr1

расписания на день 1 пара № 3

расписания на день 1 пара № 4
s6 проходит y грyппы g9 в kабинете cr1
s2 проходит y грyппы g8 в kабинете cr2
s6 проходит y грyппы g3 в kабинете cr3
s3 проходит y грyппы g2 в kабинете cr4

расписания на день 1 пара № 5
s6 проходит y грyппы g7 в kабинете cr1

расписания на день 2 пара № 1

расписания на день 2 пара № 2
s1 проходит y грyппы g9 в kабинете cr4

расписания на день 2 пара № 3

расписания на день 2 пара № 4
s2 проходит y грyппы g6 в kабинете cr1
s1 проходит y грyппы g3 в kабинете cr2
s2 проходит y грyппы g7 в kабинете cr3

расписания на день 2 пара № 5
s6 проходит y грyппы g4 в kабинете cr1
s5 проходит y грyппы g7 в kабинете cr3
s1 проходит y грyппы g2 в kабинете cr4

расписания на день 3 пара № 1
s1 проходит y грyппы g9 в kабинете cr1
s5 проходит y грyппы g3 в kабинете cr3

расписания на день 3 пара № 2
s5 проходит y грyппы g9 в kабинете cr1
s4 пр

In [939]:
state = SA_for_teachers(result, [])
print("!")

changed
step: 2, enegry : 69, temp : 100000000.0
changed
step: 3, enegry : 79, temp : 5000000000.0
changed
step: 4, enegry : 69, temp : 166666666666.6667
changed
step: 5, enegry : 68, temp : 4166666666666.667
changed
step: 6, enegry : 88, temp : 83333333333333.33
changed
step: 7, enegry : 109, temp : 1388888888888888.8
changed
step: 8, enegry : 69, temp : 1.9841269841269836e+16
changed
step: 9, enegry : 69, temp : 2.4801587301587296e+17
changed
step: 10, enegry : 70, temp : 2.7557319223985884e+18
changed
step: 11, enegry : 90, temp : 2.755731922398588e+19
changed
step: 12, enegry : 67, temp : 2.505210838544171e+20
changed
step: 13, enegry : 89, temp : 2.0876756987868092e+21
changed
step: 14, enegry : 70, temp : 1.6059043836821609e+22
changed
step: 15, enegry : 49, temp : 1.1470745597729719e+23
changed
step: 16, enegry : 28, temp : 7.647163731819813e+23
changed
step: 17, enegry : 27, temp : 4.779477332387383e+24
changed
step: 18, enegry : 46, temp : 2.8114572543455192e+25
changed
step: 

In [942]:
energy(state, result)

3

In [941]:
print_sch_w_teachers(result, state)

расписания на день 1 пара № 1

расписания на день 1 пара № 2
s2 проходит y грyппы g5 в kабинете cr1 ведет t13

расписания на день 1 пара № 3

расписания на день 1 пара № 4
s6 проходит y грyппы g9 в kабинете cr1 ведет t4
s2 проходит y грyппы g8 в kабинете cr2 ведет t2
s6 проходит y грyппы g3 в kабинете cr3 ведет t8
s3 проходит y грyппы g2 в kабинете cr4 ведет t6

расписания на день 1 пара № 5
s6 проходит y грyппы g7 в kабинете cr1 ведет t10

расписания на день 2 пара № 1

расписания на день 2 пара № 2
s1 проходит y грyппы g9 в kабинете cr4 ведет t12

расписания на день 2 пара № 3

расписания на день 2 пара № 4
s2 проходит y грyппы g6 в kабинете cr1 ведет t13
s1 проходит y грyппы g3 в kабинете cr2 ведет t15
s2 проходит y грyппы g7 в kабинете cr3 ведет t1

расписания на день 2 пара № 5
s6 проходит y грyппы g4 в kабинете cr1 ведет t13
s5 проходит y грyппы g7 в kабинете cr3 ведет t3
s1 проходит y грyппы g2 в kабинете cr4 ведет t14

расписания на день 3 пара № 1
s1 проходит y грyппы g9 в kаб